In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from statsmodels.tsa.seasonal import seasonal_decompose

from utils import plot_quantiles, plot_barplots, plot_horizontal_barplots, get_quantiles_from_values, categories_english, shops_english

---

# Loading Data

---

### Main Data File

In [223]:
train = pd.read_csv('data/sales_train.csv')
train.shape

(2935849, 6)

In [224]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


### Items Data

In [225]:
items = pd.read_csv('data/items_en.csv')
items.shape

(22170, 4)

In [226]:
items.head()

,item_name,item_id,item_category_id,item_name_en
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,!In the power of obsession (plast.) D
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,! Abbyy Finereader 12 Professional Edition Ful...
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,*** In the rays of glory (unv) D
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,*** Blue wave (univ) D
4,***КОРОБКА (СТЕКЛО) D,4,40,*** Box (glass) D


### Item Categories Data

In [227]:
item_categories = pd.read_csv('data/item_categories_en.csv')
item_categories.shape

(84, 3)

In [228]:
item_categories.head()

,item_category_name,item_category_id,item_category_name_en
0,PC - Гарнитуры/Наушники,0,PC - Headsets/Headphones
1,Аксессуары - PS2,1,Accessories - PS2
2,Аксессуары - PS3,2,Accessories - PS3
3,Аксессуары - PS4,3,Accessories - PS4
4,Аксессуары - PSP,4,Accessories - PSP


### Shops Data

In [229]:
shops = pd.read_csv('data/shops_en.csv')
shops.shape

(60, 3)

In [230]:
shops.head()

,shop_name,shop_id,shop_name_en
0,"!Якутск Орджоникидзе, 56 фран",0,Yakutsk - Ordzhonikidze
1,"!Якутск ТЦ ""Центральный"" фран",1,Yakutsk - Central Mall
2,"Адыгея ТЦ ""Мега""",2,Adygea - Mega Mall
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Balashikha - October-Kinomir
4,"Волжский ТЦ ""Волга Молл""",4,Volgograd - Volga Mall


<div class="alert alert-block alert-info">
- Train Set has 2.9 millions sales entries and 6 columns <br>
- There are 22k unique items referenced <br>
- There are 84 unique categories of items referenced <br>
- Products are distributed across 60 shops <br>

### Test Set

In [231]:
test = pd.read_csv('data/test.csv')
test.shape

(214200, 3)

In [232]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


---

# Data Exploration

Definitions :
- Counts : number of distinct items with at least one unit sold on a day
- Volumes : Counts * nb of units sold (item_cnt_day ) 
- Sales : Volumes * Price





---

### Missing Data

In [233]:
train.isnull().sum()

KeyboardInterrupt: 

<div class="alert alert-block alert-info">
No missing data

### Data Types

In [ ]:
train.info()

<div class="alert alert-block alert-info">
Dataset contains: <br>
- one datetime field<br>
- 3 integer ID fields<br>
- 2 float fields<br>

### Merge Data and add Sales

In [ ]:
data = train.merge(items, on = 'item_id', how ='left').merge(item_categories, on = 'item_category_id', how = 'left').merge(shops, on = 'shop_id', how = 'left')
data['sales'] = data['item_price']*data['item_cnt_day']
data['shop_city'] = data['shop_name_en'].str.extract('(.*) -')
data.shape

In [ ]:
data.head()

### Dates

In [ ]:
data['date'] = pd.to_datetime(data['date'], format = "%d.%m.%Y")
data['date_m'] = data['date'].dt.strftime('%Y-%m')
data['date_d'] = data['date'].dt.strftime('%Y-%m-%d')

In [ ]:
print(data['date'].min())
print(data['date'].max())

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,3))
# Yearly Counts
data['date'].dt.year.value_counts().plot.bar(ax = ax1)
ax1.set_title('Yearly Counts')
# Yearly Volumes
data.groupby(data['date'].dt.year).agg({'item_cnt_day':'sum'}).plot.bar(ax = ax2)
ax2.set_title('Yearly Volumes')
ax2.legend().set_visible(False)
ax2.set(xlabel=None)
data.groupby(data['date'].dt.year).agg({'sales':'sum'}).plot.bar(ax = ax3)
# Yearly Sales
ax3.set_title('Yearly Sales')
ax3.legend().set_visible(False)
ax3.set(xlabel=None)

<div class="alert alert-block alert-info">
- Dates range from Jan 2013 up to October 2015 <br>
- Volume trend is globally decreasing with time <br>
- Sales trend is not aligned with volume trend and has peak in 2014. <br>
- This decorrelation between Sales and Volumes could be explained either by a difference in the products sold or by a price evolution

In [ ]:
monthly_volumes = data.groupby(data['date_m']).agg({'item_cnt_day':'sum'})
monthly_sales = data.groupby(data['date_m']).agg({'sales':'sum'})

plot_barplots(monthly_volumes, monthly_sales, "Monthly Volumes", "Monthly Sales")

<div class="alert alert-block alert-info">
We can observe a seasonality with peak at year end <br>

In [ ]:
top_volumes = data.groupby(data['date_d']).agg({'item_cnt_day':'sum'}).sort_values(by = 'item_cnt_day', ascending = False).head(50)
top_sales = data.groupby(data['date_d']).agg({'sales':'sum'}).sort_values(by = 'sales', ascending = False).head(50)

plot_barplots(top_volumes, top_sales, "Top 50 Volume Dates", "Top 50 Sales Dates", share_axis = False, height = 8)

<div class="alert alert-block alert-info">
Peak dates are mostly last days of the year<br>
29-11-2013 was an extreme high in sales

In [ ]:
bottom_volumes = data.groupby(data['date_d']).agg({'item_cnt_day':'sum'}).sort_values(by = 'item_cnt_day', ascending = False).tail(50)
bottom_sales = data.groupby(data['date_d']).agg({'sales':'sum'}).sort_values(by = 'sales', ascending = False).tail(50)

plot_barplots(bottom_volumes, bottom_sales, "Bottom 50 Volume Dates", "Bottom 50 Sales Dates", share_axis = False, height = 8)

### Shops

In [ ]:
data['shop_id'].nunique()

In [ ]:
shop_volumes = data.groupby('shop_id').agg({'item_cnt_day':'sum'})
shop_sales = data.groupby('shop_id').agg({'sales':'sum'})

plot_barplots(shop_volumes, shop_sales, "Shops Global Volumes", "Shops Global Sales")

<div class="alert alert-block alert-info">
All 60 referenced shops are present in sales data <br>
There are strong discrepancies of volumes and sales between shops <br>

In [ ]:
shop_rank_volumes = data.groupby('shop_name_en').agg({'item_cnt_day':'sum'}).sort_values(by ='item_cnt_day')
shop_rank_sales = data.groupby('shop_name_en').agg({'sales':'sum'}).sort_values(by ='sales')

plot_horizontal_barplots(shop_rank_volumes, shop_rank_sales, "Shops Rank - Global Volumes", "Shops Rank - Global Sales", share_axis = False)

<div class="alert alert-block alert-info">

Shops with best volumes and / or sales are located in Moscow and St Petersbourg   
    
We can notice some difference in the ranks in volume and sales <br>
It could be explained either by <br>
    - a difference in the products sold<br>
    - a difference in the pricing <br>
    - both

In [ ]:
monthly_shop = data.groupby(['shop_name_en','date_block_num']).agg({'item_cnt_day':'sum','sales':'sum'})

shop_rank_monthly_volumes = monthly_shop.groupby('shop_name_en').agg({'item_cnt_day':'mean'}).sort_values(by ='item_cnt_day')
shop_rank_monthly_sales = monthly_shop.groupby('shop_name_en').agg({'sales':'mean'}).sort_values(by ='sales')

plot_horizontal_barplots(shop_rank_monthly_volumes, shop_rank_monthly_sales, "Shops Rank - Monthly Volumes", "Shops Rank - Monthly Sales", share_axis = False)

<div class="alert alert-block alert-info">
Looking at the average monthly figures we can observe a few shops going up in the rank (Yakutsk shops and Outbound Trade) <br>
This is probably due to a difference in the number of months with sales between shops <br>

In [ ]:
f, ax = plt.subplots(figsize=(8,15))

# Shop Volumes
data.groupby(['shop_name_en']).agg({"date_block_num" : "nunique"}).sort_values(by ='date_block_num').plot.barh(ax=ax)
ax.legend().set_visible(False)
ax.set_title('Nb of sales months per Shop')

In [ ]:
data.groupby(['shop_id']).agg({"date_block_num" : "nunique"})['date_block_num'].value_counts().sort_index(ascending = False)

<div class="alert alert-block alert-info">
Only half of the shops have sales records in all 34 months of observations <br>
10 shops have less than 10 months of data <br>
5 shops have less than 2 months of data <br>

In [ ]:
# number of selling shops for each month
f, ax = plt.subplots(figsize=(15,3))

# Shop Volumes
data.groupby(['date_m']).agg({"shop_id" : "nunique"}).plot.bar(ax=ax)
ax.legend().set_visible(False)
ax.set_title('Nb of active shops for each month')

<div class="alert alert-block alert-info">
Number of active shops is fluctuating over the period between 42 and 52

In [ ]:
monthly_shop = data.groupby(['shop_city','date_block_num']).agg({'item_cnt_day':'sum','sales':'sum'})

shop_rank_monthly_volumes = monthly_shop.groupby('shop_city').agg({'item_cnt_day':'mean'}).sort_values(by ='item_cnt_day')
shop_rank_monthly_sales = monthly_shop.groupby('shop_city').agg({'sales':'mean'}).sort_values(by ='sales')

plot_horizontal_barplots(shop_rank_monthly_volumes, shop_rank_monthly_sales, "City Rank - Monthly Volumes", "City Rank - Monthly Sales", share_axis = False, width = 10, height = 6)

#### Shops Market Share

In [ ]:
shop_volumes = data.groupby('shop_name_en').agg({'item_cnt_day':'sum'}).sort_values('item_cnt_day',ascending = False)
shop_volumes['volume_pct'] = round(shop_volumes['item_cnt_day'] / data['item_cnt_day'].sum()*100,2)
shop_volumes['volume_cum_pct'] = shop_volumes['volume_pct'].cumsum()
shop_volumes.head(20)

In [ ]:
shop_sales = data.groupby('shop_name_en').agg({'sales':'sum'}).sort_values('sales',ascending = False)
shop_sales['sales_pct'] = round(shop_sales['sales'] / data['sales'].sum()*100,2)
shop_sales['sales_cum_pct'] = shop_sales['sales_pct'].cumsum()
shop_sales.head(20)

<div class="alert alert-block alert-info">
the 4th biggest moscow shops represents 25% of the overall sales volume and 22% of sales

In [ ]:
shops.iloc[10:12]

In [ ]:
check = data.groupby(['shop_id']).agg({"date_m" : "nunique"})
check.iloc[10:12]

In [ ]:
data[data['shop_id']==11].date_m.unique()

In [ ]:
data[data['shop_id']==10].date_m.unique()

<div class="alert alert-block alert-info">
Shop ID 10 and 11 have the same name. Shop 11 only has one month of sale which is the only one without sale for shop 10. We can assume that this is the same shop.

In [ ]:
check = data.groupby(['shop_id']).agg({"date_m" : "nunique"})
shop_activity= shops.merge(check, on = 'shop_id', how = 'left')
test_shops = test.shop_id.unique()
shop_activity[shop_activity['shop_id'].isin(test_shops)]

In [ ]:
shop_activity[shop_activity['shop_id'].isin(test_shops)].date_m.value_counts()

<div class="alert alert-block alert-info">
The test set contains 42 shops of which 37 have more than 30 months of sales history. <br>
Let's look at the 5 other shops more in detail

In [ ]:
data[data['shop_id']==34].date_m.unique()

In [ ]:
data[data['shop_id']==39].date_m.unique()

In [ ]:
data[data['shop_id']==48].date_m.unique()

In [ ]:
data[data['shop_id']==49].date_m.unique()

In [ ]:
data[data['shop_id']==36].date_m.unique()

In [ ]:
data[data['shop_id']==36].date_d.unique()

<div class="alert alert-block alert-info">
All those shops had a starting activity after 2014.<br>
In particular shop 36 (Gallery Novosibirsk) only has activity on the last 15 days of observation

### Items 

In [ ]:
print(data['item_id'].nunique())
print(items['item_id'].nunique())
print(test['item_id'].nunique())

In [ ]:
all_items = set(items['item_id'].to_list())
data_items = set(data['item_id'].to_list())
missing = [item for item in all_items if item not in data_items]
len(missing)

In [ ]:
test_items = set(test['item_id'].to_list())
print(len([a for a in test_items if a in missing]))
print(len(missing) / items.shape[0])

<div class="alert alert-block alert-info">
The test set contains 363 items that are not present in training set <br>
This represents 1.6% of all referenced items <br>
We'll need a solution for that

In [ ]:
item_stats = data.groupby('item_id').agg({'item_cnt_day':'sum','sales':'sum','shop_id':'nunique','date_block_num':'nunique', 'item_price':'mean'}).reset_index()

#### Volume

In [ ]:
item_stats['item_cnt_day'].describe().astype(int)

<div class="alert alert-block alert-info">
mean volume is 167 <br>
median volume is 33 <br>
max volume is 187,642

In [ ]:
plot_quantiles(item_stats,'item_cnt_day', title='Item Volumes Quantiles')

In [ ]:
values = [1, 10, 100, 1000, 2000]
get_quantiles_from_values(item_stats, 'item_cnt_day', values, 'volume')

In [ ]:
item_stats.iloc[item_stats.item_cnt_day.idxmax()]

<div class="alert alert-block alert-info">
Item ID 20949 is the best selling Item in volume with 187,642 units sold for a total sales amount of 928,863 dollars  <br>
It is a cheap item with an average price of 4.91 Ruble <br>
This item has sales record in 31 of the 34 observation months and was sold in 53 shops.<br>
We can also notice that some items have negative volumes let's dig further

In [ ]:
data['item_cnt_day'].value_counts().sort_index().head(10)

In [ ]:
data[data['item_cnt_day']==-22]

In [ ]:
data[data['item_id']==8023].sort_values(by = ['date_d']).tail(20)

In [ ]:
data[data['item_cnt_day']==-9]

In [ ]:
data[(data['item_id']==9242) & (abs(data['item_cnt_day'])==9)]

In [ ]:
data[data['item_cnt_day']==-6]

In [ ]:
data[(data['item_id']==3732) & (abs(data['item_cnt_day'])==6) & (data['shop_id']==42)].sort_values(by = ['date_d'])

In [ ]:
data[data['item_cnt_day']==-1]

In [ ]:
data[(data['item_id']==2552) & (data['shop_id']==25)].sort_values(by = ['date_d'])

In [ ]:
data[(data['item_id']==4896) & (data['shop_id']==25)].sort_values(by = ['date_d'])

<div class="alert alert-block alert-info">
Looking at examples cases with negative number of units sold, we can observe that they always follow a sale with similar positive volume<br>
We can then assume that negative values are product returns <br>
We could think of offsetting these negative sum operations but our goal is to project monthly sales, regardless of whether they will lead to returns later on<br>
However we also observed negative value for aggregated volumes of some items

In [ ]:
bug_id = item_stats[item_stats['item_cnt_day']<1]['item_id'].to_list()
data[data['item_id'].isin(bug_id)].sort_values(by = 'item_id')

In [ ]:
test[test['item_id'].isin(bug_id)]

<div class="alert alert-block alert-info">
a few items have an aggregated negative number of units sold  <br>
We can notice that all these items are sold in 2013. We can assume that they were sales for these items prior to the start of observation period.<br>
None of these items are present in the test set so we can remove them <br>

#### Sales

In [ ]:
plot_quantiles(item_stats, 'sales', title='Item Sales Quantiles')

In [ ]:
values = [100, 1000, 10000, 100000, 1000000]
get_quantiles_from_values(item_stats, 'sales', values, 'sales')

In [ ]:
item_stats[item_stats['sales']>100000000]

<div class="alert alert-block alert-info">
Item ID 6675 is the best selling Item in sales with 10,289 units sold for a total sales amount of 219M dollars  <br>
It is an expensive item with an average price of 22,113$ <br>
This item has sales record in 24 of the 34 observation months and was sold in 532 shops.

#### Shops presence per item

In [ ]:
plot_quantiles(item_stats, 'shop_id', title='Number of shops per item - Quantiles')

In [ ]:
values = [1, 3, 5, 10, 30, 40, 50]
get_quantiles_from_values(item_stats, 'shop_id', values, 'number of shops')

### Items Price

In [ ]:
print(data['item_price'].nunique())
print(data['item_id'].nunique())

##### Before doing price statistics we will keep only one occurence of the same product price

In [ ]:
price = data.drop_duplicates(['item_id','item_price'])
price.shape

In [ ]:
price['item_price'].describe().astype(int)

<div class="alert alert-block alert-info">
mean price is 1,000 dollars <br>
median price is 399 dollars <br>
max price is around 300,000 dollars

In [ ]:
plot_quantiles(price, 'item_price', title='Item Price Quantiles')

In [ ]:
values = [50, 100, 500, 1000, 5000]
get_quantiles_from_values(price, 'item_price', values, 'price')

In [ ]:
price[price['item_price']<0]

In [ ]:
data[(data['item_id']==2973) & (data['shop_id']==32)]

<div class="alert alert-block alert-info">
1 item has negative price but we can correct this price by looking at the other sales for the same product in the same shop <br>
It is a bit tricky as the prior sale price was 2499 and the next sale price after was 1249 (looks like a 50% discount) <br>
We will use the original price which is the most frequent: 2499 <br>

In [ ]:
data.loc[data['item_price']==-1,'item_price'] = 2499

In [ ]:
item_price = price.groupby('item_id').agg({'item_price':'nunique'}).reset_index()

In [ ]:
item_price

In [ ]:
plot_quantiles(item_price, 'item_price', title='Number of different prices for an item')

In [ ]:
values = [1,2,3,5, 10, 20 ,50]
get_quantiles_from_values(item_price, 'item_price', values, 'nb of Prices')

<div class="alert alert-block alert-info">
half of the items have less than 3 different prices <br>
89% have less than 10 prices<br>
1% of the items have more than 45 different prices <br>
Price variation could be influenced by the following factors:<br>
- Date of the sale (with underlying economical context)<br>
- Shop of the sale <br>
- Potential discounts applied <br><br>

Let's dig further into this

In [ ]:
item_shop_price = price.groupby(['item_id','shop_id']).agg({'item_price':'nunique'}).reset_index()

In [ ]:
plot_quantiles(item_shop_price, 'item_price', title='Number of different prices for a couple shop - item')

In [ ]:
values = [1,2,3, 5, 10]
get_quantiles_from_values(item_shop_price, 'item_price', values, 'nb of Prices')

<div class="alert alert-block alert-info">
Looking at item price per shop, 95% of the item have less than 3 different prices<br>
Price variance is mostly explained by shops <br>

In [ ]:
high_variation = item_shop_price[item_shop_price['item_price']>20]

In [ ]:
high_variation['item_id'].value_counts()

In [ ]:
item_outlier = price[price['item_id']==17717]
item_outlier['item_price'].describe()

In [ ]:
items[items['item_id']==17717]

In [ ]:
17717 in test['item_id']

<div class="alert alert-block alert-info">
Looking at items with very high price variation on same shop (above 20), we can notice that one item is standing out (item ID 17717) <br>
This item is called "Reception of funds 1C-Online" and is under the Service Category.<br>
This is probably related to some online services provided by 1C or maybe only payment card<br>
This ID is present in the test set so we can't remove it <br>
Price for this item ranges from 9 to 16790 which is very strange

In [ ]:
high_variation['shop_id'].value_counts()

In [ ]:
shops.iloc[12]

In [ ]:
shop_outlier = item_shop_price[item_shop_price['shop_id']==12].sort_values('item_price')
shop_outlier = shop_outlier[shop_outlier['item_price']>20]
shop_outlier

In [ ]:
for item in [11365, 11369, 11370, 11371, 11372, 11373, 13753, 13754]:
    temp = price[price['item_id']==item]['item_price']
    name = items[items['item_id']==item]['item_name_en'].values[0]
    item_category = items[items['item_id']==item]['item_category_id'].values[0]
    item_category_name = item_categories[item_categories['item_category_id']==item_category]['item_category_name_en'].values[0]
    
    print('item name is {} , category is {} min price is {}, max price is {}, median price is {}'.format(name, item_category_name, int(temp.min()), int(temp.max()), int(temp.median())))

In [ ]:
item_categories[item_categories['item_category_name_en']=="Delivery of Goods"]

In [ ]:
deliveries = data[data['item_category_id']==9]
deliveries.shop_name_en.unique()

<div class="alert alert-block alert-info">
Looking at shop with very high price variation on given items (above 20), one shop is standing out (shop ID 12) <br>
This shop is the online shop <br>
It has 8 items with high variation<br>
These items are in the category delivery of goods (which is exclusive to the online shop)<br> 
They might be delivery fees<br>
These items are present in the test set so we can't remove them <br>

In [ ]:
variation = item_shop_price[item_shop_price['item_price']>5]
variation = variation.merge(items, on = 'item_id', how = 'left').merge(item_categories, on = 'item_category_id', how='left')
variation.groupby('item_category_name_en').agg({'item_id':'nunique'}).sort_values(by ='item_id', ascending = False)

In [ ]:
variation.head()

In [ ]:
check = data[(data['item_id']==31) & (data['shop_id']==31)]

In [ ]:
check.groupby('date_m').agg({'item_price':'mean','item_cnt_day':'sum'})

<div class="alert alert-block alert-info">
More generally, items with price variation above 5 are mostly movies and games <br>
Looking at a specific example we can observe that the value of these items are decreasing with time and sales volumes as well <br>
We shall consider the first date of sell for each item to assess the recency of an item <br>

### Item Categories

In [ ]:
items

In [ ]:
categories_share = round(pd.DataFrame(items.item_category_id.value_counts(normalize = True))*100,2)
categories_share['cumulative_pct'] = categories_share['item_category_id'].cumsum()
categories_share = categories_share.rename(columns = {'item_category_id':'items_pct'})
categories_share = categories_share.merge(item_categories, left_index = True, right_on = 'item_category_id', how='left')
categories_share.head(20)

<div class="alert alert-block alert-info">
There are 84 categories of items <br>
Category ID 40 (Movie - DVD) represents 22% of the referenced items <br>
Top 5 categories represent 50% of the reference items <br>
top 20 categories cover 80% of the referenced items

In [ ]:
categories_volumes = data.groupby('item_category_name_en').agg({'item_cnt_day':'sum'}).sort_values('item_cnt_day')
categories_sales = data.groupby('item_category_name_en').agg({'sales':'sum'}).sort_values('sales')

plot_horizontal_barplots(categories_volumes, categories_sales, "Global Volumes by Item Category", "Global Sales by Item Category", share_axis = False, width = 20, height = 15)

In [ ]:
categories_volumes = categories_volumes.sort_values('item_cnt_day',ascending = False)
categories_volumes['volume_pct'] = round(categories_volumes['item_cnt_day'] / data['item_cnt_day'].sum()*100,2)
categories_volumes['volume_cum_pct'] = categories_volumes['volume_pct'].cumsum()
categories_volumes = categories_volumes.merge(item_categories, left_index = True, right_on = 'item_category_name_en', how = 'left')
categories_volumes.head(20)

<div class="alert alert-block alert-info">
Category ID 40 (Movie - DVD) represents 18% of the volume of sold items<br>
Top 5 categories Includes Movies (DVD & BluRay), Games (PC & PS3) and Local Music CDs and it represent 52% of the volume of sold items <br>
top 20 categories cover 84% of the volume sold

In [ ]:
categories_sales = categories_sales.sort_values('sales',ascending = False)
categories_sales['sales_pct'] = round(categories_sales['sales'] / data['sales'].sum()*100,2)
categories_sales['sales_cum_pct'] = categories_sales['sales_pct'].cumsum()
categories_sales = categories_sales.merge(item_categories, left_index = True, right_on = 'item_category_name_en', how = 'left')
categories_sales.head(20)

<div class="alert alert-block alert-info">
Category ID 19 (Games - PS3) represents 12% of the total amount of sales<br>
Top 5 categories are Games categories and represent 46% of the sales <br>
top 20 categories cover 80% of the sales

In [ ]:
monthly_categories = data.groupby(['date_m','item_category_name_en']).agg({'item_cnt_day':'sum','sales':'sum','item_id':'nunique'}).reset_index()
monthly_categories

In [ ]:
categories_sales

In [ ]:
top_categories = categories_volumes['item_category_name_en'].head(5).to_list()

monthly_categories_top = monthly_categories[monthly_categories['item_category_name_en'].isin(top_categories)]

fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(x = "date_m", y = "item_cnt_day", hue='item_category_name_en', data = monthly_categories_top, )
plt.xticks(rotation = 90)
plt.show()

<div class="alert alert-block alert-info">
Volumes for the main categories have been decreasing strongly over the observation period

### Seasonality

#### Global seasonality

In [ ]:
monhtly_data = data.groupby(['date_m']).agg({'item_cnt_day':'sum','sales':'sum','item_id':'nunique', 'item_price':'mean'}).reset_index()
monhtly_data['date_m'] = pd.to_datetime(monhtly_data['date_m'], format = '%Y-%m')
monhtly_data.set_index('date_m', inplace=True)

In [ ]:
seasonal_volume = pd.pivot_table(monhtly_data, index=monhtly_data.index.month, columns=monhtly_data.index.year,
                    values='item_cnt_day', aggfunc='sum')

seasonal_sales = pv = pd.pivot_table(monhtly_data, index=monhtly_data.index.month, columns=monhtly_data.index.year,
                    values='sales', aggfunc='sum')

seasonal_price = pv = pd.pivot_table(monhtly_data, index=monhtly_data.index.month, columns=monhtly_data.index.year,
                    values='item_price', aggfunc='sum')

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,4))
sns.lineplot(data=seasonal_volume, ax = ax1)
ax1.set_xlabel("Months")
ax1.set_ylabel("Volume")
ax1.set_title("Sales Volum month/year")

sns.lineplot(data=seasonal_sales, ax =ax2)
ax2.set_xlabel("Months")
ax2.set_ylabel("Sales")
ax2.set_title("Sales month/year")

sns.lineplot(data=seasonal_price, ax =ax3)
ax3.set_xlabel("Months")
ax3.set_ylabel("Average Price")
ax3.set_title("Average Price month/year")

In [ ]:
decompose_result_mult = seasonal_decompose(monhtly_data['item_cnt_day'], model="multiplicative")
trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid
fig = decompose_result_mult.plot()
fig.set_size_inches((9, 6))
fig.tight_layout()
plt.show()

In [ ]:
decompose_result_mult = seasonal_decompose(monhtly_data['sales'], model="multiplicative")
trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid
fig = decompose_result_mult.plot()
fig.set_size_inches((9, 6))
fig.tight_layout()
plt.show()

In [ ]:
decompose_result_mult = seasonal_decompose(monhtly_data['item_id'], model="multiplicative")
trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid
fig = decompose_result_mult.plot()
fig.set_size_inches((9, 6))
fig.tight_layout()
plt.show()

In [ ]:
decompose_result_mult = seasonal_decompose(monhtly_data['item_price'], model="multiplicative")
trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid
fig = decompose_result_mult.plot()
fig.set_size_inches((9, 6))
fig.tight_layout()
plt.show()

<div class="alert alert-block alert-info">
The global seasonality is quite clear. We can observe peaks each year at year end and lows around summer time <br>
In terms of trend, we can observe differents: <br>
- Volumes and number of items sold are decreasing over the period <br>
- Sales have their peak in 2014 <br>
- Average Item Price is increasing    

#### Categories seasonality

In [ ]:
monthly_categories = data.groupby(['date_m','item_category_id']).agg({'item_cnt_day':'sum','sales':'sum','item_id':'nunique', 'item_price':'mean'}).reset_index()
monthly_categories['date_m'] = pd.to_datetime(monthly_categories['date_m'], format = '%Y-%m')
monthly_categories.set_index('date_m', inplace=True)

In [ ]:
# volumes per top 5 categories and special categories like Online shop and Service
for category in monthly_categories.item_category_id.unique():
    category_name = item_categories[item_categories['item_category_id']== category]['item_category_name_en'].values[0]
    print(category_name, 'volume')
    df = monthly_categories[monthly_categories['item_category_id']==category]
    if df.shape[0]==34:
        decompose_result_mult = seasonal_decompose(df['item_cnt_day'], model="multiplicative")
        trend = decompose_result_mult.trend
        seasonal = decompose_result_mult.seasonal
        residual = decompose_result_mult.resid
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(25,2))
        ax1.plot(trend)
        ax2.plot(seasonal)
        ax3.plot(residual)
        plt.show()
    else:
        print('not enough months for this category')

<div class="alert alert-block alert-info">
Volume Trends are different for each categorie <br>
It is mostly decreasing but for some categories we can also observe increasing trends 